In [1]:
!pip install pydub
!pip install scipy.io
!pip install reformer_pytorch
!pip install mamba-ssm

from google.colab import drive
import os
from pydub import AudioSegment
from scipy.io import wavfile
import numpy as np
import io
drive.mount('/content/drive')

%cd /content/drive/MyDrive/
#!git clone https://github.com/wzhwzhwzh0921/S-D-Mamba
!pip install -r ./requirements.txt


ERROR: Could not find a version that satisfies the requirement scipy.io (from versions: none)
ERROR: No matching distribution found for scipy.io
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1

In [ ]:


def convert_to_wav(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".mp3") or filename.endswith(".m4a"):
            input_path = os.path.join(directory, filename)
            output_path = os.path.join(directory, os.path.splitext(filename)[0] + ".wav")
            audio = AudioSegment.from_file(input_path)
            audio.export(output_path, format="wav")
            os.remove(input_path)

            print(f"Converted {filename} to WAV.")

def convert_to_timeseries(directory):
    os.makedirs("./S-D-Mamba/dataset/dopplerdata/", exist_ok=True)
    time_series_data = []

    for filename in os.listdir(directory):
        if filename.endswith(".wav"):
            wav_path = os.path.join(directory, filename)
            sample_rate, audio_data = wavfile.read(wav_path)
            time_series_data.append(audio_data.flatten())

    combined_time_series = np.concatenate(time_series_data)
    txt_path = os.path.join("./S-D-Mamba/dataset/dopplerdata/", "doppler.txt")
    with io.open(txt_path, 'w', encoding='utf-8') as file:
      np.savetxt(txt_path, combined_time_series, delimiter=",")
    print(f"Saved combined time series as {txt_path}")

directory = './dopplerdata/'

#convert_to_wav(directory)
convert_to_timeseries(directory)


<ipython-input-3-2be84f2888ff>:19: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, audio_data = wavfile.read(wav_path)


KeyboardInterrupt: 

In [ ]:
import pandas as pd

def convert_to_csv(directory):
    os.makedirs("./S-D-Mamba/dataset/dopplerdata/", exist_ok=True)
    time_series_data = []

    for filename in os.listdir(directory):
        if filename.endswith(".wav"):
            wav_path = os.path.join(directory, filename)
            sample_rate, audio_data = wavfile.read(wav_path)
            audio_data = audio_data.flatten()

            # Save each WAV file as a separate CSV file
            csv_filename = os.path.splitext(filename)[0] + ".csv"
            csv_path = os.path.join("./S-D-Mamba/dataset/dopplerdata/", csv_filename)
            pd.DataFrame(audio_data).to_csv(csv_path, index=False, header=False)
            print(f"Saved individual CSV file as {csv_path}")
            time_series_data.append(audio_data)

    combined_time_series = np.concatenate(time_series_data)

    # Convert the data type to a more compact representation (e.g., int16 or float32)
    combined_time_series = combined_time_series.astype(np.int16)

    combined_csv_path = os.path.join("./S-D-Mamba/dataset/dopplerdata/", "doppler_combined.csv")
    pd.DataFrame(combined_time_series).to_csv(combined_csv_path, index=False, header=False)

    print(f"Saved individual CSV files and combined time series as {combined_csv_path}")

directory = './dopplerdata/'
convert_to_csv(directory)

<ipython-input-2-a5b4f0aa6c79>:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, audio_data = wavfile.read(wav_path)


Saved individual CSV file as ./S-D-Mamba/dataset/dopplerdata/FULLSetMixed.csv
Saved individual CSV file as ./S-D-Mamba/dataset/dopplerdata/04 - Working With Nihilism - Dopplerhaus.csv
Saved individual CSV file as ./S-D-Mamba/dataset/dopplerdata/02 - Faustian Deal - Dopplerhaus.csv
Saved individual CSV file as ./S-D-Mamba/dataset/dopplerdata/Via Coober Pedy Master 24bit48kHz.csv
Saved individual CSV file as ./S-D-Mamba/dataset/dopplerdata/FINAL EP REHEARSAL_20.03.23.csv
Saved individual CSV file as ./S-D-Mamba/dataset/dopplerdata/EP LAUNCH_WAREHOUSE REHEARSAL_19.03.23.csv
Saved individual CSV file as ./S-D-Mamba/dataset/dopplerdata/bergy set rehearsal.csv
Saved individual CSV file as ./S-D-Mamba/dataset/dopplerdata/21.2.23Three Phase Rehearsal Studios 9.csv
Saved individual CSV file as ./S-D-Mamba/dataset/dopplerdata/EP LAUNCH REHEARSAL_16.03.23.csv
Saved individual CSV file as ./S-D-Mamba/dataset/dopplerdata/catholic core.csv
Saved individual CSV file as ./S-D-Mamba/dataset/dopplerdata

[Errno 2] No such file or directory: './drive/MyDrive/S-D-Mamba/'
/content/drive/MyDrive


In [ ]:
 %cd ./S-D-Mamba/
!python -u run.py --is_training 1 --root_path ./dataset/dopplerdata/ --data_path BOOMBAPBOOMBAPDemo4.11.23.csv --model_id doppler_96_720 --model S_Mamba --data doppler --features S --seq_len 441000 --pred_len 441000 --e_layers 2 --enc_in 137 --dec_in 137 --c_out 137 --des 'Exp' --d_model 512 --d_ff 512 --itr 1 --target '0'

[Errno 2] No such file or directory: './S-D-Mamba/'
/content/drive/MyDrive/S-D-Mamba
Args in experiment:
Namespace(is_training=1, model_id='doppler_96_720', model='S_Mamba', data='doppler', root_path='./dataset/dopplerdata/', data_path='BOOMBAPBOOMBAPDemo4.11.23.csv', features='S', target='0', freq='s', checkpoints='./checkpoints/', seq_len=441000, label_len=48, pred_len=441000, enc_in=137, dec_in=137, c_out=137, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=512, moving_avg=25, factor=1, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=1, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', loss='MSE', lradj='type1', use_amp=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', exp_name='MTSF', channel_independence=False, inverse=False, class_strategy='projection', target_root_path='./data/electricity/', target_data_path='electricity.csv', efficient_training=False, use_

In [ ]:
from google.colab import runtime
runtime.unassign()